# IMDb Reviews Scraper
### Iordache Ioan-Bogdan
I chose to collect movie reviews from IMDb since they contain useful metadata for us to be able to build a dataset and run different Machine Learning algorithms on it.  We may try to detect the sentiment of a review by trying to predict its score or find how useful a review was for other users. All of these can be done through supervised learning since we already have the dataset "labeled" by the metadata mentioned above, like the score and the number of users who claimed that a review was helpful and those who claimed the opposite.

## Searching for a movie
All IMDb URLs for a given movie are based on a movie ID. For a better scraping experience, I chose to implement the logic for retrieving this ID by providing the movie title (not necessarily the exact title).

In order to do that, we use the search function of IMDb through the search URL by providing the search string. We need some processing on the search string: remove non-alphanumerical characters and replace white spaces with the '+' character.

In [1]:
import re


def movie_title_to_search_string(title: str) -> str:
    """ Remove non-alphanumerical characters and convert movie
        title string to the format accepted by IMDB search link
        (i.e. replace whitespaces with +)
    """
    regex = re.compile("[^a-zA-Z 0-9]")
    title = regex.sub(" ", title)
    return re.sub("\s+", "+", title)

By searching for the first <i>td</i> tag with the <i>.result_text</i> class we are able to get the ID of the first movie returned by the search query. If that tag cannot be found we throw an exception. It is also a good idea to retrieve the official title of the movie as well.

In [2]:
import bs4
import requests
from typing import Tuple

headers = {"Accept-Language": "en-US"}


class MovieNotFoundException(Exception):
    pass


def get_movie_true_title_and_id(movie_title: str) -> Tuple[str, str]:
    """ Get IMDB id for given movie
    """
    search_string = movie_title_to_search_string(movie_title)
    response = requests.get(
        f"https://www.imdb.com/find?q={search_string}",
        headers = headers
    )
    souped_content = bs4.BeautifulSoup(response.content, "html.parser")
    try:
        first_result = souped_content.find("td", class_="result_text")
        movie_link = first_result.find("a")
        movie_title = movie_link.text
        movie_id = movie_link["href"].split('/')[2]
        return movie_title, movie_id
    except Exception as ex:
        raise MovieNotFoundException(
            f"Movie with title \'{movie_title}\' could NOT be found"
        )


In [3]:
# Test get_movie_id
get_movie_true_title_and_id("spiderman 2")

('Spider-Man 2', 'tt0316654')

## Extracting reviews
### One review from its container
A single review appears on the reviews page of a movie encapsulated inside a <i>.review-container div</i> tag. Below we implement the logic for extracting information for a single review from its container. We extract:
 * review title (always present)
 * text, the content of the review (always present)
 * score, integer between 1 and 10 (can be missing)
 * user name (always present)
 * date of the review (in the format yyyy/mm/dd)
 * number of users that voted on the usefulness of this review
 * number of users that considered this review useful

Some processing has to be done on the review text. We remove all the tags but keep the text between them. If there are any URLs present in the text, we remove them and replace them with an <b>`[URL]`</b> token. Finally, we replace all multiple consecutive white spaces with a single space character and strip all the white spaces from the beginning and the end of the text string.  

In [10]:
from datetime import datetime
from typing import List, NamedTuple, Optional


class Review(NamedTuple):
    title: str
    text: str
    score: Optional[int]
    user: str
    date: str
    helpfulness_votes: int
    positive_helpfulness_votes: int

    def __str__(self) -> str:
        return (
            f"{self.title} ({self.score}/10)\n" +
            f"by {self.user} on {self.date}\n" +
            f"{self.text}\n\n" +
            f"{self.positive_helpfulness_votes} out of " +
            f"{self.helpfulness_votes} found it useful"
        )


def process_review_text(text_div: bs4.element.Tag) -> str:
    """ Remove html tags and shrink consecutive whitespaces.
        Replace all URLs with a special token.
    """
    # remove all tags but keep the text data
    text = text_div.get_text(" ", strip=True)
    # replace links with [URL] token
    text = re.sub(r"https?://[^ ]+", "[URL]", text)
    text = re.sub(r"www.[^ ]+", "[URL]", text)
    # change multiple consecutive whitespaces into a single space
    text = re.sub("\s+", " ", text)
    return text


def extract_review_from_div(tag: bs4.element.Tag) -> Review:
    """ Extract all review info from the review-container div
    """
    title = tag.find("a", class_="title").text.strip()
    user = tag.find("span", class_="display-name-link").find("a").text
    date = tag.find("span", class_="review-date").text
    # change date format to numerical one
    date = datetime.strptime(date, "%d %B %Y").strftime("%Y/%m/%d")
    try:
        score = int(tag.find("span", class_="point-scale").previous_sibling.text)
    except Exception:
        score = None  # no score is specified for this review
    help_text = tag.find(
        "div", class_="actions text-muted"
    ).text.strip().split(' ')
    positive_helpfulness_votes, helpfulnes_votes = int(help_text[0].replace(',', '')), int(help_text[3].replace(',', ''))

    # get review text
    text_div = tag.find("div", class_="text show-more__control")
    text = process_review_text(text_div)
    
    return Review(title=title, text=text, score=score, user=user, date=date,
        helpfulness_votes=helpfulnes_votes,
        positive_helpfulness_votes=positive_helpfulness_votes
    )

We test the above methods on the code of a review container extracted from an IMDb page. Some changes were applied in order to catch some corner cases.

In [11]:
# Test extract_review_from_div

div = """<div class="review-container">
        <div class="lister-item-content">
    <div class="ipl-ratings-bar">
            <span class="rating-other-user-rating">
            <svg class="ipl-icon ipl-star-icon  " xmlns="http://www.w3.org/2000/svg" fill="#000000" height="24" viewBox="0 0 24 24" width="24">
                <path d="M0 0h24v24H0z" fill="none"/>
                <path d="M12 17.27L18.18 21l-1.64-7.03L22 9.24l-7.19-.61L12 2 9.19 8.63 2 9.24l5.46 4.73L5.82 21z"/>
                <path d="M0 0h24v24H0z" fill="none"/>
            </svg>
                <span>9</span><span class="point-scale">/10</span>
            </span>
    </div>
<a href="/review/rw5638050/?ref_=tt_urv"
class="title" > Just rewatching in 2020
</a>            <div class="display-name-date">
                    <span class="display-name-link"><a href="/user/ur68951879/?ref_=tt_urv"
>koyushun</a></span><span class="review-date">14 April 2020</span>
            </div>
            <div class="content">
                <div class="text show-more__control">I was a kid when I watched this in cinema back in 2004
I just want to say after all these years after a few version of Spider-Mans and all the MCU movie.
<br>
<br>
This one is hands down the best Superhero movie <a href="google.com">text to keep</a>.
It has everything done within 2 hours. Perfectly <a href="https://google.com">https://google.com</a> caught up what it left of from the previous Spider-Man and Toby Maguire will always be my Spider-Man <a href="www.google.com">www.google.com</a>.</div>
                <div class="actions text-muted">
                    26 out of 37 found this helpful.
                        <span>
                            Was this review helpful? <a href="/registration/signin?ref_=urv"
> Sign in</a> to vote.
                        </span>
                        <br/>
                    <a href="/review/rw5638050/?ref_=tt_urv"
>Permalink</a>
                </div>
            </div>
        </div>
        <div class="clear"></div>
    </div>
"""

div_soup = bs4.BeautifulSoup(div).find("div", class_="review-container")
print(extract_review_from_div(div_soup))


Just rewatching in 2020 (9/10)
by koyushun on 2020/04/14
I was a kid when I watched this in cinema back in 2004 I just want to say after all these years after a few version of Spider-Mans and all the MCU movie. This one is hands down the best Superhero movie text to keep . It has everything done within 2 hours. Perfectly [URL] caught up what it left of from the previous Spider-Man and Toby Maguire will always be my Spider-Man [URL] .

26 out of 37 found it useful


### Many reviews
Scraping all of the reviews for a movie it is a little bit trickier than iterating through all of the review containers on the page. The pagination on IMDb is done by clicking the "Load more" button at the bottom of the page. This button does not change the URL of the page, but instead it does an asynchronous request that fetches the new components that need to be rendered on the current page. To simulate that request we need a "pagination key" that is specified in an attribute of the load button. Using that we can retrieve more reviews and also the pagination key for the next batch.

Using this method we can scrape all the reviews but we chose to stop after a maximum of 100 for a given movie.

In [12]:
MAX_REVIEW_COUNT = 100


def scrape_reviews(movie_id: str) -> List[Review]:
    """Get all reviews for given movie id
    """
    request_link = f"https://www.imdb.com/title/{movie_id}/reviews"
    review_divs = []
    while len(review_divs) < MAX_REVIEW_COUNT:
        response = requests.get(
            request_link,
            headers = headers
        )
        souped_content = bs4.BeautifulSoup(response.content, "html.parser")
        review_divs.extend(
            souped_content.find_all("div", class_="review-container")
        )
        
        # load more link
        load_more_div = souped_content.find("div", class_="load-more-data")
        if not load_more_div:
            break
        pagination_key = load_more_div["data-key"]
        request_link = (
            f"https://www.imdb.com/title/{movie_id}/reviews/_ajax?ref_=undefined&" +
            f"paginationKey={pagination_key}"
        )
    reviews = []
    for review_div in review_divs:
        reviews.append(extract_review_from_div(review_div))

    return reviews

In [13]:
# Test scrape_reviews
import random


reviews = scrape_reviews("tt0316654")
print(len(reviews))
print(random.choice(reviews))

100
"I believe there's a hero in all of us." (9/10)
by utgard14 on 2014/05/29
Being Spider-Man is wrecking poor Peter Parker's life. He's had to give up his chance at love with Mary Jane, he can't hold a job because he's always late, his grades are suffering at college, his Aunt May is struggling with the loss of beloved Uncle Ben, and his best friend Harry blames Spider-Man for the death of his father. All of this leads to Peter deciding to give up being a superhero. But a new threat rises in the form of mad physicist Otto Octavius, dubbed Dr. Octopus by the press, that may force Peter out of retirement. Sam Raimi's direction is much improved from the first film. This one he seems completely at home with the big budget blockbuster and creates many memorable scenes and images, as he often does in his lower budget work. Comic fans should appreciate the homages to the works of Lee, Ditko, and Romita, The CGI improves from the first film. There's still a little bit of the rubbery animated

## Building a dataset
Now we are able to run these methods for a list of movie titles and retrieve all the reviews for them. We are going to store them in an in-memory DataFrame and we can also write them to disk on a CSV file for later usage.

In [14]:
from tqdm import tqdm

MOVIES = ["Spiderman 2", "taxi driver", "goodfellas", "the   green mile", "Detachment", "Horrible Bosses 2", "Death of a Salesman", "Intermission", "October Sky", "Blackfish", "Hotel Transylvania", "Moon", "Les Choristes", "Interstellar", "The Strangers", "Pulp Fiction", "The Fault in Our Stars", "Ferris Bueller's Day Of", "Primal Fear", "Crash"]
COLUMNS = ["movie", "title", "text", "score", "user", "date", "helpfulness_votes", "positive_helpfulness_votes"]


all_reviews = []
for movie in tqdm(MOVIES):
    movie_title, movie_id = get_movie_true_title_and_id(movie)
    reviews = scrape_reviews(movie_id)
    all_reviews.extend(
        (
            (
                movie_title, review.title, review.text, review.score, review.user,
                review.date, review.helpfulness_votes, review.positive_helpfulness_votes
            ) 
            for review in reviews
        )
    )

100%|██████████| 20/20 [01:25<00:00,  4.26s/it]


In [15]:
import pandas as pd
import random


random.shuffle(all_reviews)
df = pd.DataFrame(all_reviews, columns=COLUMNS)
df.to_csv("imdb_reviews.csv")
df.head(10)

,movie,title,text,score,user,date,helpfulness_votes,positive_helpfulness_votes
0,Moon,Amazing Throwback Sci-Fi,There is little that I can do by way of review...,10.0,drunkenhopfrog,2009/07/24,41,27
1,The Chorus,Marvellous - Please do not remake in English!!,I recorded this on my Sky Plus box out of curi...,10.0,beresfordjd,2006/05/01,4,3
2,The Green Mile,Crap !,Hard to believe that so many people seem to be...,1.0,Doc-Zucker,2006/04/17,137,40
3,Detachment,"A touching, thought provoking masterpiece","Wow, was not expecting that! Popped into the c...",10.0,darren-153-890810,2012/07/18,6,4
4,Detachment,Best Movie of 2011,Saw this at a local film festival with little ...,10.0,feguiza,2011/11/27,312,274
5,Pulp Fiction,Wow! Just wow!,I absolutely love this film. Around 5 or 6 yea...,10.0,destiniem,2020/04/16,9,7
6,The Chorus,Love and courage in education,Mr. Mathieu communicated with the students lik...,9.0,IreneWatsonC,2020/03/20,0,0
7,Moon,"Wow, just Wow...",Possibly the best sci-fi film I have ever seen...,10.0,caitlin_online,2010/01/03,10,5
8,The Fault in Our Stars,"A ""Sick love story""",This review is originally from my blog Derpy R...,9.0,jfischer-8-534134,2014/05/23,146,81
9,The Fault in Our Stars,Luckily...,Just what I've written in the title... LUCKILY...,1.0,yiwen-96,2015/03/05,7,4
